In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

from datetime import datetime, date, timedelta

from bokeh.layouts import layout, column, row
from bokeh.models import ColumnDataSource, GeoJSONDataSource, DateSlider, Button, Select, DataRange1d, LogColorMapper
from bokeh.plotting import figure
from bokeh.palettes import Viridis6
from bokeh.io import show, output_notebook, curdoc

from data_handler import DataHandler

output_notebook()

Loading BokehJS ...

# ToDo:

* get other data and include it to show other details (tests, economy, unemployment, ...
* fix ranges for line graphs
* link views
* change the color by day in the animation by cases
* select different countrys and when clicked we only see the data for this country

In [10]:
def app(doc):
    # load data
    dh = DataHandler()
    source = ColumnDataSource(dh.initial_view())
    
    # create line plots   
    line_plot_1 = figure(plot_width=800, 
                   plot_height=200,
                   x_axis_type="datetime")
    
    line_plot_2 = figure(plot_width=800, 
                   plot_height=200,
                   x_axis_type="datetime")

    line_plot_1.line('date', 'line_1', source=source)
    line_plot_2.line('date', 'line_2', source=source)
    
    # set inditial optins
    SELECT_OPTIONS = dh.fields
    INITIAL_VIEW = [SELECT_OPTIONS[0], SELECT_OPTIONS[1]]
    
     # date ranges
    x_range_start = datetime(dh.date_range[0].year, dh.date_range[0].month, dh.date_range[0].day)
    x_range_end = datetime(dh.date_range[1].year, dh.date_range[1].month, dh.date_range[1].day)

    line_plot_1.x_range.start = x_range_start
    line_plot_1.x_range.end = x_range_end
    line_plot_2.x_range.start = x_range_start
    line_plot_2.x_range.end = x_range_end
    
    line_plot_1.y_range.start = 0
    line_plot_2.y_range.start = 0
    line_plot_1.y_range.end = dh.y_range_end['EUR'][INITIAL_VIEW[0]]
    line_plot_2.y_range.end = dh.y_range_end['EUR'][INITIAL_VIEW[1]]
    
    # chloropleth
    TOOLS_chloropleth = 'hover,tap,pan'
    chloropleth = figure(plot_width=500, 
                         plot_height=400, 
                         tools=TOOLS_chloropleth)
    chloropleth.hover.tooltips = [
        ('Country', '@country'),
    ]
    data_europe_country = dh.data[dh.date_range[1]].groupby(['ISO3']).sum().reset_index()
    geo_data = dh.geo_data
    geo_data['color_by'] = geo_data.apply(lambda row: data_europe_country[data_europe_country['ISO3'] == row['ISO3']]['cases'].values[0], axis = 1)
    geo_source = GeoJSONDataSource(geojson=geo_data.to_json())
    palette = tuple(reversed(Viridis6))
    color_mapper = LogColorMapper(palette=palette)
    chloropleth.patches('xs', 'ys',
                        source=geo_source,
                        fill_alpha=0.7, 
                        fill_color={'field': 'color_by', 'transform': color_mapper},
                        line_color='white', 
                        line_width=0.5)
    
    # create interactions
    # slider
    def update(attrname, old, new):
        date = slider.value_as_datetime.date()
        source.data = dh.update_view(date, select1.value, select2.value)
    
    slider = DateSlider(start=dh.date_range[0], end=dh.date_range[1], value=dh.date_range[0], step=1, title="Date")
    slider.on_change('value', update)
    
    callback_id = None
    # automated play button
    def animate_update():
        global callback_id
        date = slider.value_as_datetime.date() + timedelta(days=1)
        if date >= dh.date_range[1]:
            date = dh.date_range[0]
        slider.value = date
    
    def animate():
        global callback_id
        if button.label == '► Play':
            button.label = '❚❚ Pause'
            callback_id = curdoc().add_periodic_callback(animate_update, 200)
        else:
            button.label = '► Play'
            curdoc().remove_periodic_callback(callback_id)
    
    button = Button(label='► Play', width=60)
    button.on_click(animate)
    
    # Selectors   
    OPTIONS = ['cases', 'deaths']
    select1 = Select(value=INITIAL_VIEW[0], options=SELECT_OPTIONS)
    select2 = Select(value=INITIAL_VIEW[1], options=SELECT_OPTIONS)
    
    select1.on_change('value', update)
    select2.on_change('value', update)
    
    # create layout
    tools = column(button, slider, select1, select2)
    top = row(chloropleth, tools)
    bottom = column(line_plot_1, line_plot_2)
    layout = column(top, bottom)
    
    # add to server
    doc.add_root(layout)

In [11]:
show(app)